In [1]:
import os
import glob
import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from functools import reduce


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats & ML
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
import numpy as np

# Volcano plot utility (we’ll build manually with matplotlib)
from matplotlib import cm

# Gene ID conversion
import mygene

# Pathway enrichment (KEGG, Enrichr, etc.)
import gseapy as gp
from gprofiler import GProfiler

## Introduction

Python. Using gene expression count data from control and irradiated mouse salivary gland tissues, we identify differentially expressed genes (DEGs) using the pydeseq2 package and conduct KEGG pathway enrichment analysis with g:Profiler. The aim is to characterize transcriptomic changes resulting from radiation exposure and uncover the biological pathways most affected. Where appropriate, results are compared to those generated using the R-based DESeq2 and clusterProfiler pipeline to assess consistency across platforms.

## 1. Data Import

In this step, we load raw gene count files for three untreated (control) and three irradiated samples. Each file contains raw Ensembl gene counts.

In [2]:
# 1. Load & Merge Count Files
raw_data_path = "../data/raw/"
sample_files = sorted(glob.glob(os.path.join(raw_data_path, "GSM*.txt")))
sample_names = ["UT1", "UT2", "UT3", "IR1", "IR2", "IR3"]

df_list = []
for f, name in zip(sample_files, sample_names):
    df = pd.read_csv(f, sep="\t", header=None, names=["Ensembl", name])
    df_list.append(df)

merged_counts = reduce(lambda left, right: pd.merge(left, right, on="Ensembl"), df_list)
merged_counts.set_index("Ensembl", inplace=True)


In [3]:
# 2. Metadata
meta = pd.DataFrame({
    "condition": ["Control"] * 3 + ["Irradiated"] * 3
}, index=sample_names)

### Notes on Input and Output Files

- **Input**: RNA-seq read count files are stored in the `../data/raw/` directory. Each file represents a single biological replicate and contains raw Ensembl gene counts. Files are named using the pattern `GSM*.txt` (e.g., `GSM1.txt`, `GSM2.txt`, etc.), and are read into R using `read_tsv()`.

- **Output**:
  - Differential expression results: `../results/deseq2_results_python.csv`
  - KEGG enrichment results: `../results/kegg_enrichment_python.csv`
  - Figures:
    - Volcano plot: `../figures/volcano_plot_python.png`
    - KEGG barplot: `../figures/kegg_barplot_python.png`
    - Heatmap of top variable genes: `../figures/heatmap_top30_genes_python.png`
    - PCA plot: `../figures/pca_plot_python.png`
    - Sample distance heatmap: `../figures/sample_distance_heatmap_python.png`


## 2. DESeq2 Differential Expression Analysis

In [4]:
# 3. Run DESeq2
dds = DeseqDataSet(
    counts=merged_counts.T,
    metadata=meta,
    design_factors="condition"
)
dds.deseq2()

C:\Users\nadee\AppData\Local\Temp\ipykernel_49240\2315759797.py:2: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(
Fitting size factors...
... done in 0.01 seconds.



Using None as control genes, passed at DeseqDataSet initialization


Fitting dispersions...
... done in 1.01 seconds.

Fitting dispersion trend curve...
... done in 0.38 seconds.

Fitting MAP dispersions...
... done in 0.96 seconds.

Fitting LFCs...
... done in 1.32 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.



In [5]:
# 4. Get Results
ds = DeseqStats(dds, contrast=["condition", "Irradiated", "Control"])
ds.summary()

res_df = ds.results_df.copy()
res_df.index.name = "Ensembl"
res_df.reset_index(inplace=True)

# 5. Filtering significant DEGs (padj < 0.05, abs(log2FC) > 1)
filtered_degs = res_df[
    (res_df["padj"] < 0.05) & (res_df["log2FoldChange"].abs() > 1)
].copy()

# 6. Save outputs
res_df.to_csv("../results/deseq2_results_python.csv", index=False)
filtered_degs.to_csv("../results/deseq2_significant_genes_python.csv", index=False)

# Optional: Show top genes
print(filtered_degs[["Ensembl", "log2FoldChange", "padj"]].head(10))


Running Wald tests...
... done in 2.27 seconds.



Log2 fold change & Wald test p-value: condition Irradiated vs Control
                            baseMean  log2FoldChange     lfcSE      stat  \
Ensembl                                                                    
ENSMUSG00000000001      4.638397e+02        0.239703  0.151017  1.587265   
ENSMUSG00000000003      0.000000e+00             NaN       NaN       NaN   
ENSMUSG00000000028      2.118693e+01        0.751958  0.458837  1.638836   
ENSMUSG00000000031      9.182366e+01       -0.928921  0.609175 -1.524884   
ENSMUSG00000000037      1.334689e+00       -0.639686  1.912022 -0.334560   
...                              ...             ...       ...       ...   
__no_feature            9.084230e+05       -0.529336  0.108097 -4.896847   
__ambiguous             1.692785e+06       -0.154816  0.118391 -1.307674   
__too_low_aQual         3.168626e+05       -0.865697  1.706863 -0.507186   
__not_aligned           9.125217e+05        0.320248  0.448384  0.714228   
__alignment_not_un

## 3. Volcano Plot of DEGs

In [18]:
# Load DESeq2 results
res_df = pd.read_csv("../results/deseq2_results_python.csv")
res_df["padj"].fillna(1, inplace=True)

# Use gProfiler to map Ensembl IDs to symbols
gp = GProfiler(return_dataframe=True)
conversion = gp.convert(organism='mmusculus', query=res_df["Ensembl"].tolist())

# Merge symbol info
res_df = res_df.merge(conversion[['incoming', 'name']], left_on='Ensembl', right_on='incoming', how='left')
res_df.rename(columns={"name": "symbol"}, inplace=True)
res_df.drop(columns=["incoming"], inplace=True)

# Define EnhancedVolcano-style categories
conditions = [
    (res_df["padj"] >= 0.05) & (abs(res_df["log2FoldChange"]) <= 1),
    (res_df["padj"] >= 0.05) & (abs(res_df["log2FoldChange"]) > 1),
    (res_df["padj"] < 0.05) & (abs(res_df["log2FoldChange"]) <= 1),
    (res_df["padj"] < 0.05) & (abs(res_df["log2FoldChange"]) > 1)
]
labels = ["NS", "Log2 FC", "p-value", "p-value and log2 FC"]
res_df["significant"] = np.select(conditions, labels, default="NS")


# Color palette matching EnhancedVolcano
palette = {
    "NS": "gray",
    "Log2 FC": "green",
    "p-value": "blue",
    "p-value and log2 FC": "red"
}

# Create Volcano plot
plt.figure(figsize=(10, 7))
ax = sns.scatterplot(
    data=res_df,
    x="log2FoldChange",
    y=-np.log10(res_df["padj"]),
    hue="significant",
    palette=palette,
    linewidth=0,
    alpha=0.7
)

# Threshold lines
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)
plt.axvline(1, color='black', linestyle='--', linewidth=1)
plt.axvline(-1, color='black', linestyle='--', linewidth=1)

# Annotate top 10 significant genes (with symbols)
top_genes = res_df.sort_values("padj").dropna(subset=["symbol"]).head(10)
for _, row in top_genes.iterrows():
    ax.text(row["log2FoldChange"], -np.log10(row["padj"]), row["symbol"],
            fontsize=8, ha='right' if row["log2FoldChange"] < 0 else 'left',
            va='center', color='black')

# Plot decorations
plt.title("Volcano Plot: Irradiated vs Control")
plt.xlabel("log2 Fold Change")
plt.ylabel("-log10 Adjusted p-value")
plt.legend(title=None)  # remove legend title to match R style
plt.tight_layout()
plt.savefig("../figures/volcano_plot_python.png", dpi=300)
plt.close()

## 4. Map Ensembl IDs to Entrez IDs

In [7]:
# Initialize mygene client
mg = mygene.MyGeneInfo()

# Extract unique Ensembl IDs
ensembl_ids = res_df["Ensembl"].dropna().unique().tolist()

# Query mygene.info for Entrez IDs
query_result = mg.querymany(
    ensembl_ids,
    scopes="ensembl.gene",
    fields="entrezgene",
    species="mouse",
    as_dataframe=True
)

# Drop rows without Entrez mapping
query_result = query_result[~query_result["entrezgene"].isna()]

# Clean and prepare for merging
query_result = query_result[["entrezgene"]]
query_result.reset_index(inplace=True)
query_result.rename(columns={"query": "Ensembl", "entrezgene": "entrez"}, inplace=True)

# Merge with DESeq2 results
res_df = res_df.merge(query_result, on="Ensembl", how="left")

# Filter significant DEGs with Entrez
sig_entrez_ids = res_df.loc[
    (res_df["padj"] < 0.05) & (res_df["log2FoldChange"].abs() > 1) & res_df["entrez"].notnull(),
    "entrez"
].astype(str).tolist()

# Save mapped data (optional)
res_df.to_csv("../results/deseq2_results_with_entrez_python.csv", index=False)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
3 input query terms found dup hits:	[('ENSMUSG00000072694', 2), ('ENSMUSG00000106441', 2), ('ENSMUSG00000112376', 2)]
4068 input query terms found no hit:	['ENSMUSG00000000325', 'ENSMUSG00000004613', 'ENSMUSG00000011052', 'ENSMUSG00000021745', 'ENSMUSG000


## 5. KEGG Pathway Enrichment

Using enrichKEGG, we identify biological pathways enriched among significantly differentially expressed genes.

In [8]:
gp = GProfiler(return_dataframe=True)

results = gp.profile(
    organism='mmusculus',
    query=sig_entrez_ids,  # or gene symbols
    sources=['GO:BP', 'KEGG']
)

results.to_csv("../results/gprofiler_enrichment_results_python.csv", index=False)

## 6. Visualize Top KEGG Pathways

This barplot displays the top 10 enriched KEGG pathways based on adjusted p-values.

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Sort and select top 10 KEGG pathways
top_kegg = results[results['source'] == 'KEGG'].sort_values("p_value").head(10).copy()
top_kegg["Gene Count"] = top_kegg["intersection_size"]
top_kegg["log_pval"] = -np.log10(top_kegg["p_value"])

# Normalize colors
norm = mcolors.Normalize(vmin=top_kegg["log_pval"].min(), vmax=top_kegg["log_pval"].max())
cmap = plt.cm.coolwarm
colors = cmap(norm(top_kegg["log_pval"].values))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(
    y=top_kegg["name"],
    width=top_kegg["Gene Count"],
    color=colors,
    edgecolor="black"
)
ax.set_xlabel("Gene Count")
ax.set_title("Top Enriched KEGG Pathways")
ax.invert_yaxis()

# Attach colorbar to the current axis
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("-log10(p-value)")

# Save and show
plt.tight_layout()
plt.savefig("../figures/kegg_barplot_python.png", dpi=300)
plt.close()

## 7. Heatmap of Highly Variable Genes

We display a heatmap of the top 30 most variable genes across all samples to visualize sample clustering and expression patterns.

In [22]:
# 1. Get VST counts from dds
dds.vst()
vst_counts = dds.layers["vst_counts"]

# 2. Create DataFrame with genes as rows, samples as columns
vst_df = pd.DataFrame(
    vst_counts.T,
    index=dds.var_names,
    columns=dds.obs_names
)

# 3. Select top 30 most variable genes
top_genes = vst_df.var(axis=1).sort_values(ascending=False).head(30).index

# 4. Subset and row-center the matrix
mat = vst_df.loc[top_genes]
mat_centered = mat.sub(mat.mean(axis=1), axis=0)

# 5. Plot clustered heatmap
cg = sns.clustermap(
    mat_centered,
    cmap="RdYlBu_r",
    metric="correlation",
    method="average",
    col_cluster=True,
    row_cluster=True,
    linewidths=0.5,
    xticklabels=True,
    yticklabels=True,
    figsize=(10, 10)
)

# 6. Set a proper title using fig.suptitle and adjust layout
cg.fig.suptitle("Top 30 Most Variable Genes", fontsize=16)
cg.fig.subplots_adjust(top=0.92)  # make space for title

# 7. Save and show
cg.savefig("../figures/heatmap_top30_genes_python.png")
plt.close()

Fit type used for VST : parametric
Using None as control genes, passed at DeseqDataSet initialization


Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 1.26 seconds.



In [20]:
# 1. Scale the data
scaled_data = StandardScaler().fit_transform(vst_df.T)  # Transpose to shape (samples x genes)

# 2. Run PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# 3. Create PCA dataframe
pca_df = pd.DataFrame(pca_result, columns=["PC1", "PC2"], index=vst_df.columns)
pca_df["condition"] = dds.obs["condition"].values

# 4. Plot
group_colors = {
    "Control": "#F8766D",     # Pink/red-ish
    "Irradiated": "#00BFC4"   # Teal/cyan
}

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="condition", s=100, palette=group_colors)

pc1_var = pca.explained_variance_ratio_[0] * 100
pc2_var = pca.explained_variance_ratio_[1] * 100
plt.xlabel(f"PC1: {pc1_var:.1f}% variance")
plt.ylabel(f"PC2: {pc2_var:.1f}% variance")
plt.title("PCA of Sample Gene Expression")
plt.legend(title="Condition")
plt.tight_layout()
plt.savefig("../figures/pca_plot_python.png")
plt.close()

### Interpretation of Quality Control Results

The PCA plot of variance-stabilized gene expression data reveals a clear separation between control and irradiated samples along the first two principal components. Principal Component 1 (PC1) explains 26.2% of the total variance, while PC2 accounts for an additional 21.5%, together capturing nearly half of the total variation in the data.

Samples from the control group (UT1–UT3) cluster tightly on the left side of the plot, while irradiated samples (IR1–IR3) are distinctly positioned on the right, suggesting that radiation exposure induces substantial transcriptomic changes. This separation supports the biological relevance of the experimental conditions and provides a strong foundation for downstream differential expression analysis.

Although the sample sizes are small, the consistent grouping by condition indicates good data quality and minimal technical variation, strengthening confidence in the results.